In [1]:
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn

In [10]:
DEVICE = 'cuda' # 'cuda' or 'cpu'

NUM_CLASSES = 19   

BATCH_SIZE = 16                     
LR = 0.05    # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5  # Regularization, you can keep this at the default
NUM_EPOCHS = 5   # Total number of training epochs (iterations over dataset)
STEP_SIZE = 20    # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down
LOG_FREQUENCY = 5

In [11]:
from Cityscapes import Cityscapes
normalizer = transforms.Normalize(mean=(0.285, 0.323, 0.282), std=(0.5, 0.5, 0.5))

train_transform = transforms.Compose([transforms.Resize((512, 256)), transforms.ToTensor()])
eval_transform = transforms.Compose([transforms.Resize((512, 256)), transforms.ToTensor()])

train_dataset = Cityscapes('data/', partition_type='A', split='train', transform=train_transform)
test_dataset = Cityscapes('data/', partition_type='A', split='val', transform=eval_transform)

#i, l = train_dataset.__getitem__(1)
#plt.imshow(l)

In [12]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [13]:
from bisenetv2 import BiSeNetV2
!pip install torchinfo
from torchinfo import summary

net = BiSeNetV2(3, output_aux=False, pretrained=False)

In [14]:
# Define loss function
criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy

parameters_to_optimize = net.parameters() # In this case we optimize over all the parameters of AlexNet

# Define optimizer
optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

# Define scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

In [22]:
import torch
def mIoU(y_true, y_pred, numClass):
    correct_pred = torch.zeros(numClass) # int = (mesh : M, wire : W and background : B)
    den = torch.zeros(numClass) # den = M + W + B = (M or W or B) + (M and W and B) + ..

    for i in range (y_true.shape[0]):
        for j in range(y_true.shape[1]):
            for k in range(y_true.shape[2]):
                if (y_pred[i][j][k])==(y_true[i][j][k]):
                    correct_pred[(y_true[i][j][k])]+=1
                den[(y_pred[i][j][k])] += 1
                den[(y_true[i][j][k])] += 1
    mIoU = 0
    IoU_classes = []
    for i in range(numClass):
        if den[i]!=0:
            IoU =correct_pred[i]/(den[i]-correct_pred[i])
            IoU_classes.append(IoU)

    mIoU=sum(IoU_classes)/len(IoU_classes)
    return mIoU


In [23]:
# By default, everything is loaded to cpu
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

#cudnn.benchmark # Calling this optimizes runtime

current_step = 0
# Start iterating over the epochs
for epoch in range(NUM_EPOCHS):
    print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_lr()))
    
    # Iterate over the dataset
    for images, labels in train_dataloader:
        # Bring data over the device of choice
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        net.train() # Sets module in training mode

        # PyTorch, by default, accumulates gradients after each backward pass
        # We need to manually set the gradients to zero before starting a new iteration
        optimizer.zero_grad() # Zero-ing the gradients

        # Forward pass to the network
        outputs = net(images)
        #print(outputs.size())

        # Compute loss based on output and ground truth
        loss = criterion(outputs, labels)
        mIoU = mIoU(outputs, labels, NUM_CLASSES)
        # Log loss
        if current_step % LOG_FREQUENCY == 0:
            print('Step {}, Loss {}'.format(current_step, loss.item()))

        # Compute gradients for each layer and update weights
        loss.backward()  # backward pass: computes gradients
        optimizer.step() # update weights based on accumulated gradients
        mIoU.step()
        current_step += 1

    # Step the scheduler
    scheduler.step() 

Starting epoch 1/5, LR = [0.05]


RuntimeError: Boolean value of Tensor with more than one value is ambiguous